<a href="https://colab.research.google.com/github/bagustris/ravdess_song/blob/main/ravdess_song_sd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition

In [ ]:
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
!git clone https://github.com/bagustris/ravdess_song.git

--2021-09-07 02:26:40--  https://www.dropbox.com/s/l5hjh95gabnubn8/archive.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/l5hjh95gabnubn8/archive.zip [following]
--2021-09-07 02:26:40--  https://www.dropbox.com/s/raw/l5hjh95gabnubn8/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5b9fc6c504c8d0cfe120264ef5.dl.dropboxusercontent.com/cd/0/inline/BVpQ7pPbyXMSVda8EEwnLlY0M_cU5cQcfFLOylAE8Bs-J2mInbPncuKjcOVTQdBzTZlheNJPJqg86Ut0ueZ_8m2oOdWfx5S3iEUjNMjcyBcWb8FuLX9I0digHcUwOzm1kuI5yBSnsK4FryUUuqoV5Gnv/file# [following]
--2021-09-07 02:26:40--  https://uc5b9fc6c504c8d0cfe120264ef5.dl.dropboxusercontent.com/cd/0/inline/BVpQ7pPbyXMSVda8EEwnLlY0M_cU5cQcfFLOylAE8Bs-J2mInbPncuKjcOVTQdBzTZlheNJPJqg86Ut0ueZ_8m2oOdWfx

In [ ]:
cd ravdess_song/

Archive:  archive.zip
   creating: archive/
   creating: archive/Actor_01/
  inflating: archive/Actor_01/03-02-01-01-01-01-01.wav  
  inflating: archive/Actor_01/03-02-01-01-01-02-01.wav  
  inflating: archive/Actor_01/03-02-01-01-02-01-01.wav  
  inflating: archive/Actor_01/03-02-01-01-02-02-01.wav  
  inflating: archive/Actor_01/03-02-02-01-01-01-01.wav  
  inflating: archive/Actor_01/03-02-02-01-01-02-01.wav  
  inflating: archive/Actor_01/03-02-02-01-02-01-01.wav  
  inflating: archive/Actor_01/03-02-02-01-02-02-01.wav  
  inflating: archive/Actor_01/03-02-02-02-01-01-01.wav  
  inflating: archive/Actor_01/03-02-02-02-01-02-01.wav  
  inflating: archive/Actor_01/03-02-02-02-02-01-01.wav  
  inflating: archive/Actor_01/03-02-02-02-02-02-01.wav  
  inflating: archive/Actor_01/03-02-03-01-01-01-01.wav  
  inflating: archive/Actor_01/03-02-03-01-01-02-01.wav  
  inflating: archive/Actor_01/03-02-03-01-02-01-01.wav  
  inflating: archive/Actor_01/03-02-03-01-02-02-01.wav  
  inflating: 

In [ ]:
# list all files in the dataset
data_path = 'archive'
files = glob.glob(os.path.join(data_path + '/*/', '*.wav'))
files.sort()

In [ ]:
# check by showing the first file
files[0]

'archive/Actor_01/03-02-01-01-01-01-01.wav'

In [ ]:
# function to extract feature
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name, sr=None)
    stft = np.abs(librosa.stft(X))
    mfcc = np.mean(librosa.feature.mfcc(
        y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    mfcc_std = np.std(librosa.feature.mfcc(
        y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(
        S=stft, sr=sample_rate).T, axis=0)
    chroma_std = np.std(librosa.feature.chroma_stft(
        S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    mel_std = np.std(librosa.feature.melspectrogram(
        X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(
        S=stft, sr=sample_rate).T, axis=0)
    contrast_std = np.std(librosa.feature.spectral_contrast(
        S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(
        y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
    tonnetz_std = np.std(librosa.feature.tonnetz(
        y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
    return (mfcc, chroma, mel, contrast, tonnetz,
            mfcc_std, chroma_std, mel_std, contrast_std, tonnetz_std)

In [ ]:
# create empty list to store features and labels
feat = []
lab = []

In [ ]:
# iterate over all files
for file in files:
    print("processing ...", file)
    feat_i = np.hstack(extract_feature(file))
    lab_i = os.path.basename(file).split('-')[2]
    feat.append(feat_i)
    lab.append(int(lab_i)-1)  # make labels start from 0

processing ... archive/Actor_01/03-02-01-01-01-01-01.wav
processing ... archive/Actor_01/03-02-01-01-01-02-01.wav
processing ... archive/Actor_01/03-02-01-01-02-01-01.wav
processing ... archive/Actor_01/03-02-01-01-02-02-01.wav
processing ... archive/Actor_01/03-02-02-01-01-01-01.wav
processing ... archive/Actor_01/03-02-02-01-01-02-01.wav
processing ... archive/Actor_01/03-02-02-01-02-01-01.wav
processing ... archive/Actor_01/03-02-02-01-02-02-01.wav
processing ... archive/Actor_01/03-02-02-02-01-01-01.wav
processing ... archive/Actor_01/03-02-02-02-01-02-01.wav
processing ... archive/Actor_01/03-02-02-02-02-01-01.wav
processing ... archive/Actor_01/03-02-02-02-02-02-01.wav
processing ... archive/Actor_01/03-02-03-01-01-01-01.wav
processing ... archive/Actor_01/03-02-03-01-01-02-01.wav
processing ... archive/Actor_01/03-02-03-01-02-01-01.wav
processing ... archive/Actor_01/03-02-03-01-02-02-01.wav
processing ... archive/Actor_01/03-02-03-02-01-01-01.wav
processing ... archive/Actor_01

In [ ]:
len(feat)
len(lab)

1012

In [ ]:
# assign hsf to X, lab to y; reshape X for LSTM
X = np.array(feat)
y = np.array(lab)

In [ ]:
# reshape x untuk lstm
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [ ]:
# split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
# functio to build LSTM model}
def model_lstm():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model


In [ ]:
# create the model
model = model_lstm()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 1, 386)            1544      
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 256)            658432    
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
lstm_5 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                

In [ ]:
# train the LSTM model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/200
57/57 [==============================] - 7s 24ms/step - loss: 1.4271 - accuracy: 0.4275
Epoch 2/200
57/57 [==============================] - 1s 23ms/step - loss: 0.6580 - accuracy: 0.7527
Epoch 3/200
57/57 [==============================] - 1s 25ms/step - loss: 0.4720 - accuracy: 0.8187
Epoch 4/200
57/57 [==============================] - 1s 25ms/step - loss: 0.3527 - accuracy: 0.8725
Epoch 5/200
57/57 [==============================] - 1s 24ms/step - loss: 0.3206 - accuracy: 0.8747
Epoch 6/200
57/57 [==============================] - 1s 24ms/step - loss: 0.2498 - accuracy: 0.9044
Epoch 7/200
57/57 [==============================] - 1s 24ms/step - loss: 0.2380 - accuracy: 0.9099
Epoch 8/200
57/57 [==============================] - 1s 24ms/step - loss: 0.2024 - accuracy: 0.9308
Epoch 9/200
57/57 [==============================] - 1s 24ms/step - loss: 0.1899 - accuracy: 0.9341
Epoch 10/200
57/57 [==============================] - 1s 23ms/step - loss: 0.1536 - accuracy: 0.9495

In [ ]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

7/7 [==============================] - 0s 6ms/step - loss: 0.1225 - accuracy: 0.9608
Loss:  0.12246515601873398 --> Accuracy:  0.9607843160629272
